<a href="https://colab.research.google.com/github/Santosh-Gupta/Research2Vec2/blob/master/PublicResearch2vec15MWRegStart_9_15_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Sep 15 20:57:23 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy
import random 
import hashlib

import torch.optim as optim
from tqdm import tqdm
import sys
import tensorflow as tf
import collections
import numpy as np

from collections import deque
numpy.random.seed(12345)

use_cuda = True
# if use_cuda and torch.cuda.is_available():
#     net.cuda()

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
import hashlib
import math
import os
import random
import sys
from tempfile import gettempdir
import zipfile
import h5py
import requests
import pickle

# import loguniform
# from loguniform import LogUniform

data_index = 0
vocabulary_size = 14886544+1 

In [0]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [0]:
#from StylesSantosh
download_file_from_google_drive('1-1_aIpBG1kusXIO5w4Z81khFIoil69q7', 'SemanticScholar14MFinal.h5')


In [5]:
#From StylesSantosh gdrive 
h5f2 = h5py.File('SemanticScholar14MFinal.h5','r')
trainData = np.column_stack(( h5f2['labels'+str(0) ][:] , h5f2['batch'+str(0) ][:] ))
for i in range(1, 235):
    if i%10==0:
        print(i)
    toAppend2 = np.column_stack(( h5f2['labels'+str(i) ][:] , h5f2['batch'+str(i) ][:] ))
    trainData = np.concatenate((trainData, toAppend2))


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230


In [6]:
trainData

array([[0,
        array([ 4928722,  3922609, 14413953, 10103423,  8948498], dtype=int32)],
       [1,
        array([12557217,  5572869, 13415223,  2532000, 14609022,  9830632,
        9800679,  7504595, 10752682], dtype=int32)],
       [2,
        array([10458710,  7176517, 10268240,  4173086,  8617671,  4674075,
       12580461,  2434641,  3694004,  9734870,  1314108,  8879955,
        6468499, 12092464,  2962425, 13680848, 10590392, 10203584,
       12816205,  7484678,  7985600, 12896218, 14882024,  6783345,
         969850, 10709191,  4541728,  4312270,  6174902,   530425,
        4843145,  4838613, 11404068,  9900162, 10578750, 12955180,
        4602929,  4097386,  8870275,  7518195, 11849786,  2947773,
       11653892,  7599644,  5895991,  1381764,  5853764, 11048535,
       14128229, 11490202,   954680, 11998906,  9196156,  4506953,
        6597761,  7034485,  3008940,  9816877,  1748801, 10159466,
        2745090, 14842579,   788308,  5984365], dtype=int32)],
       ...,
     

In [7]:
len(trainData)

14886543

In [0]:
del h5f2
del toAppend2

In [0]:
data_index = 0
epoch_index = 0
recEpoch_indexA = 0 #Used to help keep store of the total number of epoches with the models

def generate_batch(batch_size, inputCount, negRate): #batch size = number of labels
  #inputCount = number of inputs per label
    global data_index, epoch_index
    
    batch = np.ndarray(shape=(batch_size, inputCount), dtype=np.int32) 
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    negs = np.zeros(shape=(batch_size, negRate), dtype=np.int32)
    
    n=0
    while n < batch_size:
        if len( set(trainData[data_index, 1]) ) >= inputCount:
            labels[n,0] = trainData[data_index, 0]
            batch[n] = random.sample( set(trainData[data_index, 1]),  inputCount)
            negs[n] = np.random.randint(vocabulary_size-1, size=(negRate))
            n = n+1
            data_index = (data_index + 1) % len(trainData) #may have to do something like len my_data[:]
            if data_index == 0:
                epoch_index = epoch_index + 1
                print('Completed %d Epochs' % epoch_index)
        else:
            data_index = (data_index + 1) % len(trainData)
            if data_index == 0:
                epoch_index = epoch_index + 1
                print('Completed %d Epochs' % epoch_index)
    
    return batch, labels, negs    
      


In [10]:
here, goes, everything = generate_batch(20, 4, 64 ) # to do next, insert %len(headernumber)
print('batch', here)
print('labels', goes)
print('negz', everything)

batch [[ 8948498  4928722 14413953 10103423]
 [ 2532000  7504595  5572869 13415223]
 [ 6597761  6783345  4541728  9816877]
 [ 4827272  4921599  7784960 11880992]
 [ 7023031  2846719  3092258 11002632]
 [13839635  2150715  3603537  7809030]
 [ 5977433 14788864  1919631  8601049]
 [ 9892408  8394768  7632793  8676396]
 [ 2698343  7744032 11733103  8441841]
 [12893030   900066  8420575 13210535]
 [ 1339707  1306472   678845  7569515]
 [ 7220937 13548319  3747359  9803338]
 [ 9523923 12100596 10697939 11615708]
 [11737196 10995808  6768877  7063482]
 [14623117 10136028  9097163  8335481]
 [14044866  7718879 10322151   219571]
 [ 5566911  6068411 10124958  8706402]
 [ 3377369 12811871  1869876   272482]
 [ 7167337  7740683 14323956  7163772]
 [11426641  9215494  4963905   763113]]
labels [[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 9]
 [10]
 [11]
 [12]
 [13]
 [14]
 [15]
 [16]
 [17]
 [19]
 [20]
 [21]]
negz [[14757349  7735425  1396132 ...  4416910 12101787 11235672]
 [ 9695200  5052139

In [0]:
class SkipGramModel(nn.Module):
    """Skip gram model of word2vec.
    Attributes:
        emb_size: Embedding size.
        emb_dimention: Embedding dimention, typically from 50 to 500.
        u_embedding: Embedding for center word.
        v_embedding: Embedding for neibor words.
    """

    def __init__(self, emb_size, emb_dimension):
        """Initialize model parameters.
        Apply for two embedding layers.
        Initialize layer weight
        Args:
            emb_size: Embedding size.
            emb_dimention: Embedding dimention, typically from 50 to 500.
        Returns:
            None
        """
        super(SkipGramModel, self).__init__()
        self.emb_size = emb_size
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=False).cuda()
        self.v_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=False).cuda()
        # self.targets = torch.ones(34816).cud
        self.init_emb()

    def init_emb(self):
        """Initialize embedding weight like word2vec.
        The u_embedding is a uniform distribution in [-0.5/em_size, 0.5/emb_size], and the elements of v_embedding are zeroes.
        Returns:
            None
        """
        initrange = (2.0 / (vocabulary_size + self.emb_dimension)) ** 0.5 # Xavier init
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.normal_(mean=0, std=math.sqrt(initrange))
        self.lossFunction = nn.BCEWithLogitsLoss( reduction = 'none' )
        

    def forward(self, pos_u, pos_v, neg_v, targets ):
        """Forward process.
        As pytorch designed, all variables must be batch format, so all input of this method is a list of word id.
        Args:
            pos_u: list of center word ids for positive word pairs.
            pos_v: list of neibor word ids for positive word pairs.
            neg_u: list of center word ids for negative word pairs.
            neg_v: list of neibor word ids for negative word pairs.
        Returns:
            Loss of this process, a pytorch variable.
        """
        emb_u = self.u_embeddings(pos_u)
        emb_v = self.v_embeddings(pos_v)
        neg_emb_v = self.v_embeddings(neg_v)

        scorePos = torch.bmm(emb_u, torch.transpose(emb_v, 1, 2)).squeeze()
        scoreNeg = torch.bmm( emb_u , torch.transpose(neg_emb_v, 1, 2) ).squeeze()

        #reduce to dot products for each set, and concatinate all the losses
        #noticed that the sign change for the negative sample dot products
        totalScores = torch.cat( ( scorePos.reshape(scorePos.shape[0], -1) , -scoreNeg.reshape(scoreNeg.shape[0], -1) ) , dim=1)

        indLoss = self.lossFunction( totalScores, targets )
        rowSum = torch.sum(indLoss, dim=1) #Sum all losses for each set
        finalLoss = torch.mean(rowSum) #Average losses across batches 

        return finalLoss

    def save_embedding(self, id2word, file_name, use_cuda):
        """Save all embeddings to file.
        As this class only record word id, so the map from id to word has to be transfered from outside.
        Args:
            id2word: map from word id to word.
            file_name: file name.
        Returns:
            None.
        """
        if use_cuda:
            embedding = self.u_embeddings.weight.cpu().data.numpy()
        else:
            embedding = self.u_embeddings.weight.data.numpy()
        fout = open(file_name, 'w')
        fout.write('%d %d\n' % (len(id2word), self.emb_dimension))
        for wid, w in id2word.items():
            e = embedding[wid]
            e = ' '.join(map(lambda x: str(x), e))
            fout.write('%s %s\n' % (w, e))


In [0]:
output_file_name = 'theOutpule.file'
batch_size = 512
window_size = 4
negSamp = 132
iterationsMax = 100000 #200001
initial_lr =1.0

num_classes = vocabulary_size
emb_dimension = 96

skip_gram_model = SkipGramModel(vocabulary_size, emb_dimension)
use_cuda = torch.cuda.is_available()

if use_cuda:
    skip_gram_model.cuda()

optimizer = optim.SGD(
    skip_gram_model.parameters(),  lr=0.01, momentum=0.9 )

In [13]:
runningLoss = 0
batch_size = 512
negSamp = 132
window_size = 4

targets = torch.ones( batch_size, window_size + negSamp , dtype = torch.float32 ).cuda()

for i in range(100000):

    batch, labels, negz = generate_batch(batch_size=batch_size, inputCount=window_size, negRate= negSamp)

    batchTensor = torch.from_numpy(batch)
    LabelTensor = torch.from_numpy(labels)
    negTensor = torch.from_numpy(negz)

    pos_u = Variable(torch.LongTensor(LabelTensor.long()))
    pos_v = Variable(torch.LongTensor(batchTensor.long()))
    neg_v = Variable(torch.LongTensor(negTensor.long()))

    if use_cuda:
        pos_u = pos_u.cuda()
        pos_v = pos_v.cuda()
        neg_v = neg_v.cuda()

    optimizer.zero_grad()

    loss = skip_gram_model.forward(pos_u, pos_v, neg_v, targets)
    runningLoss = runningLoss + loss.data.item()

    loss.backward()
    optimizer.step()

    if i%1000 == 0:
        print('i is ', i)
        print('loss is ', runningLoss/1000)
        runningLoss = 0

RuntimeError: ignored

# After training recommended lines

In [0]:
# Delete data to free up RAM 

del trainData


In [0]:
# Normalize embedings, set up a few random papers, and look for similiar papers

normalizedU = np.divide( numpyU, np.sqrt( np.sum(np.square(numpyU), axis=1) )[:,None] )

testIndexes = np.array(range(20, 100)) 
TestEmbeds = normalizedSum[testIndexes]
simNumpy = np.matmul(TestEmbeds, normalizedSum.T)

In [0]:
# Get dictionaries

download_file_from_google_drive('1-BVhAO252myZW2U6-iJ-v3vxr6zaASv3', 'Index2Title.pickle')
download_file_from_google_drive('1-BAaws04G0LH_o-L23LVtj0QSpn4yEY5', 'Index2ID.pickle')

with open('Index2Title.pickle', 'rb') as handle:
    Index2Title = pickle.load(handle)

with open('Index2ID.pickle', 'rb') as handle:
    Index2ID = pickle.load(handle)


In [0]:
Index2Title[333]

'Extent and modulation of junctional communication between pancreatic acinar cells in vivo.'

In [0]:
Index2ID[333]

'6ce85560385a968959f3ac28af40a27abc03137c'

In [0]:
print( 'Link is https://www.semanticscholar.org/paper/%s' % Index2ID[333])

Link is https://www.semanticscholar.org/paper/6ce85560385a968959f3ac28af40a27abc03137c


In [0]:
# Look at similiar papers  

for i in range(len(testIndexes)):
    paper = Index2Title[i]
    top_k = 80  # number of nearest neighbors
    nearest = (-simNumpy[i, :]).argsort()[1:top_k + 1]
    log_str = 'Nearest to %s:' % paper
    for k in xrange(top_k):
        close_paper = Index2Title[nearest[k]]
        linkID = Index2ID[nearest[k]]
        log_str = ' %s %s,' % (log_str, close_paper)
        log_str = '%s Link https://www.semanticscholar.org/paper/%s  ' % (log_str, linkID)
    print(log_str)